In [33]:
!hdfs dfs -ls /

Found 4 items
drwx------   - mapred hadoop          0 2025-04-15 18:32 /hadoop
drwxrwxrwt   - hdfs   hadoop          0 2025-04-15 18:32 /tmp
drwxrwxrwt   - hdfs   hadoop          0 2025-04-16 17:52 /user
drwxrwxrwt   - hdfs   hadoop          0 2025-04-15 18:32 /var


In [34]:
ls

bible.txt  hello.txt  HW2.ipynb  sites.csv  Untitled1.ipynb


In [35]:
!hdfs dfs -ls /user/files_for_hw

Found 2 items
-rw-r--r--   1 ubuntu hadoop    4047392 2025-04-16 18:01 /user/files_for_hw/bible.txt
-rw-r--r--   1 ubuntu hadoop   36443383 2025-04-16 18:05 /user/files_for_hw/sites.csv


In [36]:
ls


bible.txt  hello.txt  HW2.ipynb  sites.csv  Untitled1.ipynb


In [37]:
! hdfs dfs -ls /user


Found 3 items
drwxr-xr-x   - ubuntu hadoop          0 2025-04-16 18:05 /user/files_for_hw
drwxr-xr-x   - hive   hadoop          0 2025-04-15 18:32 /user/hive
drwxr-xr-x   - ubuntu hadoop          0 2025-04-15 18:35 /user/ubuntu


In [38]:
!hadoop fs -put bible.txt /user/files_for_hw

put: `/user/files_for_hw/bible.txt': File exists


In [39]:
!hadoop fs -put sites.csv /user/files_for_hw

put: `/user/files_for_hw/sites.csv': File exists


In [40]:
!hdfs dfs -ls /

Found 4 items
drwx------   - mapred hadoop          0 2025-04-15 18:32 /hadoop
drwxrwxrwt   - hdfs   hadoop          0 2025-04-15 18:32 /tmp
drwxrwxrwt   - hdfs   hadoop          0 2025-04-16 17:52 /user
drwxrwxrwt   - hdfs   hadoop          0 2025-04-15 18:32 /var


In [41]:
%%bash


hdfs dfs -ls /

Found 4 items
drwx------   - mapred hadoop          0 2025-04-15 18:32 /hadoop
drwxrwxrwt   - hdfs   hadoop          0 2025-04-15 18:32 /tmp
drwxrwxrwt   - hdfs   hadoop          0 2025-04-16 17:52 /user
drwxrwxrwt   - hdfs   hadoop          0 2025-04-15 18:32 /var


In [42]:
%%writefile mapper.py

import sys

def mapper():
    for line in sys.stdin:
        word_start = None
        for i, char in enumerate(line):
            if char.isspace():
                if word_start is not None:
                    word = line[word_start:i]
                    if len(word) > 4:
                        print(f"{word}\t1")
                    word_start = None
            elif word_start is None:
                word_start = i
        # Обработка последнего слова в строке (если строка не заканчивается пробелом)
        if word_start is not None:
            word = line[word_start:]
            if len(word) > 4:
                print(f"{word}\t1")

if __name__ == "__main__":
    mapper()

Writing mapper.py


In [49]:
%%writefile reducer.py
import sys

def reducer():
    current_word = None
    current_count = 0
    print(f"word\t count")
    for line in sys.stdin:
        word, count = line.strip().split('\t')
        count = int(count)
        
        if word == current_word:
            current_count += count
        else:
            if current_word is not None:
                print(f"{current_word}\t{current_count}")
            current_word = word
            current_count = count
    
    if current_word is not None:
        print(f"{current_word}\t{current_count}")

if __name__ == "__main__":
    reducer()

Overwriting reducer.py


In [50]:
%%bash
## пример запуска скриптов на неймноде для проверки их работы
cat bible.txt | python3 mapper.py | sort -k1,1 | python3 reducer.py

word	 count
Aaron	205
Aaron,	95
Aaron.	11
Aaron:	2
Aaron;	5
Aaron?	1
Aaronites,	2
Aaron's	31
Abaddon,	1
Abagtha,	1
Abarim,	4
abase	3
abase.	1
abased,	1
abased;	3
abasing	1
abated	4
abated.	2
Abba,	3
Abdeel,	1
Abdi,	3
Abdiel,	1
Abdon	5
Abdon,	3
Abednego,	10
Abednego.	3
Abednego:	1
Abednego;	1
Abel,	4
Abel.	2
Abel:	1
Abelbethmaachah,	1
Abelmaim,	1
Abelmeholah	1
Abelmeholah,	2
Abelmizraim,	1
Abelshittim	1
Abez,	1
abhor	18
Abhor	1
abhorred	14
abhorred,	2
abhorrest	2
abhorreth	3
abhorreth,	1
abhorreth.	1
abhorring	1
Abia:	1
Abia;	1
Abiah	1
Abiah,	1
Abiah.	1
Abiah:	1
Abialbon	1
Abiasaph:	1
Abiathar	22
Abiathar,	5
Abiathar.	2
Abiathar:	1
Abiathar's	1
Abib,	1
Abib.	1
Abib:	1
Abib;	1
Abida,	1
Abidah,	1
Abidan	5
abide	72
abide,	3
abide.	1
abide:	1
Abide	5
abideth	29
abideth.	1
abiding	8
abiding.	1
Abiel	1
Abiel,	1
Abiel.	1
Abiezer	4
Abiezer,	2
Abiezer?	1
Abiezrite:	1
Abiezrites.	2
Abigail	12
Abigail,	3
Abigail.	1
Abigail:	1
Abihail	3
Abihail,	2
Abihail:	1
Abihu	3
Abihu,	9
Abihud,	1
Abijah	12
Abi

In [54]:
!hdfs dfs -mkdir /user/files_for_hw/word_count_task

In [55]:
!hdfs dfs -ls /user/files_for_hw

Found 3 items
-rw-r--r--   1 ubuntu hadoop    4047392 2025-04-16 18:01 /user/files_for_hw/bible.txt
-rw-r--r--   1 ubuntu hadoop   36443383 2025-04-16 18:05 /user/files_for_hw/sites.csv
drwxr-xr-x   - ubuntu hadoop          0 2025-04-16 20:05 /user/files_for_hw/word_count_task


In [70]:
%%bash
## шаблон для запуска MR таски

# обязательная чистка директории, куда будем складывать результат отрабоки mr
hdfs dfs -rm -r /user/files_for_hw/word_count_task || true

# запус mr таски с указанием пути до нужного jar
hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="word-count" \
    -files ./mapper.py,./reducer.py \
    -mapper "python3 mapper.py" \
    -reducer "python3 reducer.py" \
    -input /user/files_for_hw/bible.txt \
    -output /user/files_for_hw/word_count_task

Deleted /user/files_for_hw/word_count_task
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob473077722827516808.jar tmpDir=null


2025-04-16 21:16:02,926 INFO client.RMProxy: Connecting to ResourceManager at rc1b-dataproc-m-ifco7qks15ipul9r.mdb.yandexcloud.net/10.129.0.28:8032
2025-04-16 21:16:03,157 INFO client.AHSProxy: Connecting to Application History server at rc1b-dataproc-m-ifco7qks15ipul9r.mdb.yandexcloud.net/10.129.0.28:10200
2025-04-16 21:16:03,208 INFO client.RMProxy: Connecting to ResourceManager at rc1b-dataproc-m-ifco7qks15ipul9r.mdb.yandexcloud.net/10.129.0.28:8032
2025-04-16 21:16:03,208 INFO client.AHSProxy: Connecting to Application History server at rc1b-dataproc-m-ifco7qks15ipul9r.mdb.yandexcloud.net/10.129.0.28:10200
2025-04-16 21:16:03,467 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1744824679132_0013
2025-04-16 21:16:03,976 INFO mapred.FileInputFormat: Total input files to process : 1
2025-04-16 21:16:04,198 INFO mapreduce.JobSubmitter: number of splits:30
2025-04-16 21:16:04,388 INFO mapreduce.JobSubmitter: Submitting 

In [71]:
%%bash
## запустите эту команду, чтобы вывести счетчик определенных слов, которые мы указали на grep
## Это нам будет необходимо для визуального анализа результата работы вашего скрипта
## в sort можете указать тот разделитель колонок, с которым у вас результат выплевывает редьюсер

hdfs dfs -cat /user/files_for_hw/word_count_task/* | grep  -E 'lord\.|god\.|pray\.' | sort -t$'\t' -k2.2nr  | head -n 3

lord.	17
pray.	5
